In [19]:
import torch
import cv2

# Load the model
model_path = 'enet_b2_8_best.pt'
model = torch.load(model_path, map_location='cpu')
model.eval()

# Setup video capture
video_path = 'SmileVideo.mp4'
cap = cv2.VideoCapture(video_path)

# Get video properties for output
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Setup video writer
output_video_path = 'output.mp4'
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

frame_counter = 0
frames_to_skip = fps  # Processing once every second

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_counter % frames_to_skip == 0:
        processed_frame = cv2.resize(frame, (224, 224))
        processed_frame = processed_frame.transpose((2, 0, 1))
        processed_frame = torch.tensor(processed_frame).unsqueeze(0).float()

        with torch.no_grad():
            output = model(processed_frame)
        
        points = output.squeeze(0).tolist()  # Get the list of points
        
        # Assuming pairs of values are coordinates
        for i in range(0, len(points), 2):
            x = int(points[i] * frame_width / 224)
            y = int(points[i+1] * frame_height / 224)
            cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)  # Draw points

    out.write(frame)
    frame_counter += 1

cap.release()
out.release()
cv2.destroyAllWindows()
print(f"Processed video saved to {output_video_path}")



Output Shape: torch.Size([1, 8])
Output Content: tensor([[ 232.3313,  180.9600,   37.8255,  213.8181, -178.1004,  -71.3660,
          -30.6373,  203.8099]])
Type of output: <class 'torch.Tensor'>
Output as a list of bounding boxes (example): [232.33128356933594, 180.9600372314453, 37.82545852661133, 213.81805419921875, -178.1004180908203, -71.36602020263672, -30.63728904724121, 203.80992126464844]
Unexpected model output format
Output Shape: torch.Size([1, 8])
Output Content: tensor([[327.8158, 311.2038, -41.1335, 218.2484, 140.2382,  61.2424,  22.9557,
         252.9971]])
Type of output: <class 'torch.Tensor'>
Output as a list of bounding boxes (example): [327.8158264160156, 311.2038269042969, -41.13349151611328, 218.24844360351562, 140.2381591796875, 61.24235534667969, 22.9556884765625, 252.9971466064453]
Unexpected model output format
Output Shape: torch.Size([1, 8])
Output Content: tensor([[281.6394, 281.9778,  12.4519, 227.7272, 155.3319,  16.5825, 132.2956,
         229.6538]])
